# CODIGO ANTIGUO. TENDRE´QUE ELIMINAR ESTE FICHERO

## Simple Example org LSTMv

The idea of this approach is that the model has to be able to represent the same log after being transfered into a low dimension. Then, if we train the model with the normal logs (the majority of logs in our dataset) we would have trained the model to represent the normal logs. Therefore, if an anomalous log is presented, the model would work badly, and here is where we can identify the anomaly.

In [1]:
# imports:

import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from skipgram import *
from torch.nn.utils.rnn import pad_sequence

###Step 1: Preprocessing

In [2]:
# Assuming you have a CSV file named 'logs.csv' with columns ['date', 'petition', 'URL', 'status', 'referer', 'user-agent']
# Load the CSV file into a pandas DataFrame
logs_df = pd.read_csv('../data/sitges_access_clean.csv')
#logs_df=logs_df[:2345]
logs_df.head()

,URL,bytes,referer,user-agent,group,elapsed,IP_oct0,IP_oct1,IP_oct2,IP_oct3,...,minute_cos,petition_-,petition_GET,petition_HEAD,petition_POST,petition_other,status_1,status_2,status_3,status_4
0,/comu/fancybox/jquery.fancybox.css?v=2.1.7 HTT...,-0.597348,https://www.sitgesanytime.com/es/pl415/blog/ll...,Mozilla/5.0 (compatible; AhrefsBot/7.0; +http:...,551,0.00000,0.427313,-0.342599,-0.744068,1.620095,...,1.0,0,1,0,0,0,False,1,0,0
1,/nl/pl383/ontdekken/recensies-en-gastheren/men...,0.418239,-,PhxBot/0.1 (phxbot@protonmail.com),1675,1.94591,-1.820773,-1.110110,2.453028,-0.878825,...,1.0,0,1,0,0,0,False,0,0,1
2,/comu/js/jquery-ui-1.13.1/jquery-ui.min.js HTT...,1.804745,https://www.sitgesanytime.com/es/blog.htm,"Mozilla/5.0 AppleWebKit/537.36 (KHTML, like Ge...",3121,6.21060,-1.283917,1.652930,-0.635692,-0.464640,...,1.0,0,1,0,0,0,False,1,0,0
3,/plantilles/turisme/css/font/Intro-Black-Caps....,0.808723,https://www.sitgesanytime.com/es/pl409/blog/id...,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,3340,0.00000,-1.065819,0.240710,0.104878,1.482033,...,1.0,0,1,0,0,0,False,1,0,0
4,/comu/js/jquery-ui-1.13.1/jquery-ui.min.js HTT...,1.747754,https://www.sitgesanytime.com/en/pl355/explore...,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,3487,0.00000,-0.981935,1.008220,2.145962,-1.582941,...,1.0,0,1,0,0,0,False,1,0,0


In [3]:
print(logs_df[["status_1"]].head())

   status_1
0     False
1     False
2     False
3     False
4     False


In [4]:
# Divide the data into features (X)
print(logs_df.columns)

# Convert boolean column to integers
logs_df['status_1'] = logs_df['status_1'].astype(int)

print(logs_df[["status_1"]].head())

#logs_df = logs_df.drop(["status_1", "status_5"], axis=1)


Index(['URL', 'bytes', 'referer', 'user-agent', 'group', 'elapsed', 'IP_oct0',
       'IP_oct1', 'IP_oct2', 'IP_oct3', 'month_sin', 'month_cos', 'day_sin',
       'day_cos', 'weekday_sin', 'weekday_cos', 'hour_sin', 'hour_cos',
       'minute_sin', 'minute_cos', 'petition_-', 'petition_GET',
       'petition_HEAD', 'petition_POST', 'petition_other', 'status_1',
       'status_2', 'status_3', 'status_4'],
      dtype='object')
   status_1
0         0
1         0
2         0
3         0
4         0


In [5]:
ROOT_DIR = os.path.dirname(os.path.abspath(""))

# Load the embeddings
embeddings_url = load_embeddings(os.path.join(ROOT_DIR, "models", "embeddings-url.pt"))
# Load the idx2word. This is the vocabulary where each token is associated with an index
idx2word_url = load_idx2word(os.path.join(ROOT_DIR, "models", "idx2word-url.json"))
# Load the tokenizer. Just specify the name `charbpe-url` and it will load the tokenizer, which is saved
# in the files `charbpe-url-vocab.json` and `charbpe-url-merges.txt`
tokenizer_url = load_tokenizer(os.path.join(ROOT_DIR, "models"), "charbpe-url")

url_embeddings = extract_embeddings(
	sequence = logs_df["URL"],
	embeddings = embeddings_url,
	idx2word = idx2word_url,
	tokenizer = tokenizer_url
)

embeddings_referer = load_embeddings(os.path.join(ROOT_DIR, "models/embeddings-referer.pt"))
idx2word_referer = load_idx2word(os.path.join(ROOT_DIR, "models/idx2word-referer.json"))
tokenizer_referer = load_tokenizer(os.path.join(ROOT_DIR, "models"), "charbpe-referer")
embeddings_referer.shape, embeddings_referer.mean(), embeddings_referer.std()

# --- this will take additional 3.3 GB of memory---
referers_embeddings = extract_embeddings(
	sequence = logs_df["referer"],
	embeddings = embeddings_referer,
	idx2word = idx2word_referer,
	tokenizer = tokenizer_referer
)

embeddings_useragent = load_embeddings(os.path.join(ROOT_DIR, "models/embeddings-useragent.pt"))
idx2word_useragent = load_idx2word(os.path.join(ROOT_DIR, "models/idx2word-useragent.json"))
tokenizer_useragent = load_tokenizer(os.path.join(ROOT_DIR, "models"), "charbpe-useragent")
embeddings_useragent.shape, embeddings_useragent.mean(), embeddings_useragent.std()

# --- this will take additional 3.3 GB of memory---
useragents_embeddings = extract_embeddings(
	sequence = logs_df["user-agent"],
	embeddings = embeddings_useragent,
	idx2word = idx2word_useragent,
	tokenizer = tokenizer_useragent
)

100%|██████████| 132967/132967 [00:19<00:00, 6809.33it/s]


In [6]:
# take the output column
#!out = df["level"] I OMIITED THIS
# drop the columns that are not needed
logs_df = logs_df.drop(columns=["URL", "referer", "user-agent", "group"])

In [7]:
urls=[]
referers=[]
usernames=[]

for url, referer, username in zip(url_embeddings, referers_embeddings, useragents_embeddings):
    urls.append(url.mean(0).float().numpy())
    referers.append(referer.mean(0).float().numpy())
    usernames.append(username.mean(0).float().numpy())



In [8]:
# ensure that the columns are in the correct order
logs_df = logs_df.reindex(columns=['bytes','elapsed', 'IP_oct0', 'IP_oct1', 'IP_oct2', 'IP_oct3', 'month_sin',
       'month_cos', 'day_sin', 'day_cos', 'weekday_sin', 'weekday_cos',
       'hour_sin', 'hour_cos', 'minute_sin', 'minute_cos', 'petition_-',
       'petition_GET', 'petition_HEAD', 'petition_POST', 'petition_other',
       'status_1', 'status_2', 'status_3', 'status_4'])

In [9]:
#Add paddings

"""max_length_url = logs_df['url_embeddings'].apply(len).max() 
max_length_referer = logs_df['referers_embeddings'].apply(len).max() 
max_length_useragent = logs_df['useragents_embeddings'].apply(len).max() 

print(max_length_url, max_length_referer, max_length_useragent)""" 

def convert_to_sequence(list_sequence):
    # Convert sequences to PyTorch tensors
    sequences = [torch.tensor(seq, dtype=torch.float32) for seq in list_sequence]

    # Padding sequences
    padded_sequences = pad_sequence(sequences, batch_first=True, padding_value=0.0)

    # Convert padded sequences back to DataFrame
    padded_df = pd.DataFrame(padded_sequences.numpy())

    return padded_df

def add_sequence_to_dataframe(logs_df, df, column_after):
    df1_part1 = logs_df.iloc[:, :logs_df.columns.get_loc(column_after)]
    df1_part2 = logs_df.iloc[:, logs_df.columns.get_loc(column_after):]

    # Concatenate the parts with df2 in between
    logs_df = pd.concat([df1_part1, df, df1_part2], axis=1)
    return logs_df



sequenced_urls=convert_to_sequence(urls)
sequenced_referers=convert_to_sequence(referers)
sequenced_usernames=convert_to_sequence(usernames)
print(logs_df.columns)
# Add the sequences to the original dataframe
logs_df=add_sequence_to_dataframe(logs_df, sequenced_urls, "bytes")

logs_df=add_sequence_to_dataframe(logs_df, sequenced_referers, "elapsed")
logs_df=add_sequence_to_dataframe(logs_df, sequenced_usernames, "elapsed")

#Puede que si paso las columnas del dataframe en vez de las listas me deje hacerlo
#hAECER pca en la funcion original. O no, nose
#! Mirar porque esto va mal!!!


Index(['bytes', 'elapsed', 'IP_oct0', 'IP_oct1', 'IP_oct2', 'IP_oct3',
       'month_sin', 'month_cos', 'day_sin', 'day_cos', 'weekday_sin',
       'weekday_cos', 'hour_sin', 'hour_cos', 'minute_sin', 'minute_cos',
       'petition_-', 'petition_GET', 'petition_HEAD', 'petition_POST',
       'petition_other', 'status_1', 'status_2', 'status_3', 'status_4'],
      dtype='object')


In [10]:
print(logs_df.columns)

Index([               0,                1,                2,                3,
                      4,                5,                6,                7,
                      8,                9,
       ...
           'minute_cos',     'petition_-',   'petition_GET',  'petition_HEAD',
        'petition_POST', 'petition_other',       'status_1',       'status_2',
             'status_3',       'status_4'],
      dtype='object', length=115)


In [11]:
"""['url_embeddings', 'bytes', 'referer_embeddings', 'useragent_embeddings', 'elapsed', 'IP_oct0', 'IP_oct1', 'IP_oct2', 'IP_oct3', 'month_sin',
       'month_cos', 'day_sin', 'day_cos', 'weekday_sin', 'weekday_cos',
       'hour_sin', 'hour_cos', 'minute_sin', 'minute_cos', 'petition_-',
       'petition_GET', 'petition_HEAD', 'petition_POST', 'petition_other',
       'status_1', 'status_2', 'status_3', 'status_4']"""

"['url_embeddings', 'bytes', 'referer_embeddings', 'useragent_embeddings', 'elapsed', 'IP_oct0', 'IP_oct1', 'IP_oct2', 'IP_oct3', 'month_sin',\n       'month_cos', 'day_sin', 'day_cos', 'weekday_sin', 'weekday_cos',\n       'hour_sin', 'hour_cos', 'minute_sin', 'minute_cos', 'petition_-',\n       'petition_GET', 'petition_HEAD', 'petition_POST', 'petition_other',\n       'status_1', 'status_2', 'status_3', 'status_4']"

In [12]:
# Split the data into train, validation, and test sets
X_train, X_test = train_test_split(logs_df, test_size=0.2, random_state=42)
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,minute_cos,petition_-,petition_GET,petition_HEAD,petition_POST,petition_other,status_1,status_2,status_3,status_4
41781,-0.150609,0.367842,0.150074,0.824405,-0.535935,-0.462607,-0.138367,0.442962,-0.613921,-0.783932,...,-0.743145,0,1,0,0,0,0,0,1,0
89004,-0.281218,0.121318,-0.219708,-0.012162,-0.453734,-0.163140,-0.286839,0.306187,-0.065458,-0.394805,...,0.406737,0,1,0,0,0,0,1,0,0
16853,-0.113392,-0.449912,0.116516,0.283125,-0.351581,-0.623929,-0.198384,0.210719,-0.426656,-1.319338,...,-0.406737,0,1,0,0,0,0,1,0,0
43588,-0.204459,-0.452729,-0.250437,0.000545,-0.562697,-0.235387,-0.136437,0.316499,-0.521289,-1.079770,...,0.809017,0,1,0,0,0,0,1,0,0
38249,-0.185353,0.076407,-0.249526,0.252661,-0.090764,0.047137,-0.446690,0.157244,-0.152935,-0.061733,...,1.000000,0,1,0,0,0,0,1,0,0


In [13]:
class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe.values.astype(np.float32) # Assuming dataframe is a pandas DataFrame

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        return torch.tensor(sample)

In [14]:
train_dataset = CustomDataset(X_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = CustomDataset(X_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

###Step 3: Model

In [21]:
# Define LSTM Autoencoder Model
class LogAnomalyDetector(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LogAnomalyDetector, self).__init__()
        self.encoder = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.decoder = nn.LSTM(input_size=hidden_size, hidden_size=input_size, num_layers=num_layers, batch_first=True)

    def forward(self, x):
        _, (hidden, cell) = self.encoder(x)
        output, _ = self.decoder(hidden)
        return output[-1] #Return just the last layer

input_size=len(logs_df.columns)
hidden_size=input_size #//4
num_layers=3

# Create an instance of the model
model = LogAnomalyDetector(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers)

# Print model summary
print(model)

LogAnomalyDetector(
  (encoder): LSTM(115, 115, num_layers=3, batch_first=True)
  (decoder): LSTM(115, 115, num_layers=3, batch_first=True)
)


In [16]:
# Preguntar al Eric si lo ha hecho con embeddings i si es así que como lo ha hecho para que no le de runtime errors
# Hacer la mean como solucion parche i justificarlo con lo de que al reves consumia mucha memoria

# Probar con el modelo de autoencodeer y el modelo LSTM

###Step 4: Train the model

In [17]:
# Function for training the model
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, inputs in tqdm(enumerate(train_loader, 0)):
            inputs = inputs.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), inputs.squeeze())

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader)}")



In [18]:
# Sample log entry
log_entry = test_dataset[34]

# Convert the log entry to a tensor
log_tensor = torch.tensor(log_entry).unsqueeze(0)  # Add batch dimension

# Pass the log tensor through the model
output_log_tensor = model(log_tensor)

# Convert the output tensor back to a numpy array
#output_log_entry = output_log_tensor.squeeze().detach().numpy()

# Output the reconstructed log entry
print("Reconstructed Log Entry:")
print(output_log_tensor)
print(log_tensor)

C:\Users\arroc\AppData\Local\Temp\ipykernel_5036\2349361552.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  log_tensor = torch.tensor(log_entry).unsqueeze(0)  # Add batch dimension


Reconstructed Log Entry:
tensor([ 0.0096,  0.0364, -0.0361, -0.0004,  0.0346, -0.0051, -0.0152, -0.0599,
         0.0415, -0.0331,  0.0056, -0.0353, -0.0134,  0.0452,  0.0074,  0.0175,
        -0.0513, -0.0137, -0.0013,  0.0411,  0.0479, -0.0068, -0.0094,  0.0371,
         0.0021,  0.0138,  0.0380, -0.0547, -0.0096, -0.0098, -0.0045, -0.0909,
         0.0285, -0.0152, -0.0065,  0.0448, -0.0102,  0.0363, -0.0212,  0.0094,
         0.0056, -0.0161, -0.0268, -0.0375,  0.0694,  0.0027,  0.0038,  0.0444,
        -0.0258, -0.0232,  0.0048,  0.0025,  0.0399,  0.0507,  0.0148,  0.0434,
         0.0133, -0.0137,  0.0130,  0.0662, -0.0032, -0.0471, -0.0597, -0.0207,
        -0.0107,  0.0331, -0.0098,  0.0283,  0.0524,  0.0416,  0.0636, -0.0391,
         0.0669, -0.0633, -0.0157, -0.0436, -0.0418,  0.0016, -0.0164, -0.0093,
         0.0144, -0.0200,  0.0388, -0.0389, -0.0358,  0.0182, -0.0452, -0.0088,
        -0.0526, -0.0052, -0.0530, -0.0477,  0.0152, -0.0361, -0.0246, -0.0294,
        -0.0040

In [22]:
# Define the criterion (loss function)
criterion = nn.MSELoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adjust learning rate as needed

# Train the model
num_epochs = 20

train_loss = train_model(model, train_loader, criterion, optimizer, num_epochs=num_epochs)

3325it [00:59, 56.18it/s]


Epoch [1/20], Loss: 0.126004369205102


2928it [00:52, 56.23it/s]


KeyboardInterrupt: 

### Step 5: Test the model

In [ ]:
# Test the model
def test_model(model, test_loader, criterion):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()  # Set model to evaluation mode
    model.to(device)
    test_loss = 0.0

    with torch.no_grad():
        for inputs in test_loader:
            inputs = inputs.to(device)

            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), inputs.squeeze())

            test_loss += loss.item()

    avg_test_loss = test_loss / len(test_loader)
    print(f"Average Test Loss: {avg_test_loss}")

In [ ]:
test_loss = test_model(model, test_loader, criterion)

Average Test Loss: 0.4015493282976632


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([2, 26])) that is different to the input size (torch.Size([26])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [ ]:
# To get the scoring system, compute the similarity between the output vector and the input vector. (This is just an idea)
# An anomaly should give low similarity (This is if the model performs accuratelly the task of autoencode the same vector)